**Integrantes:**

***
* Santiago Niño
* Esteban Rodriguez
* Daniel Velásquez
* Laura Sofía Ojeda

##**Simulación Basada en Agentes Colas *M/M/1/INF/INF* y *M/M/1/K/INF***

###**Instrucciones:**

* Utilizar el concepto de modelo basado en Agentes para la simular dos tipos de colas:
  * M/M/1/INF/INF
  * M/M/1/K/INF


* Realizar el análisis numérico para cada Cola y encontrar una expresión para:
  * NS= numero de usuarios
  * TS= tiempo en el sistema
  * Nq= numero de usuarios en cola
  * Ts= tiempo en cola.


* Comparar los resultados numéricos con los resultados simulados

* ¿Cómo se comporta el sistema si se aumenta el numero de servidores?

##***Desarrollo:***

Para M/M/1/INF/INF:

***

* *Número de usuarios*

\begin{equation}
NS = \lambda \times TS
\end{equation}

* *Tiempo en el sistema*
\begin{equation}
TS = \frac{1}{\mu - \lambda}
\end{equation}

* *Número de usuarios en la cola*
\begin{equation}
N_q = \frac{\lambda^2}{2\mu(\mu - \lambda)} - 1
\end{equation}

* *Tiempo en cola*
\begin{equation}
Ts = \frac{N_q}{\lambda}
\end{equation}

Para M/M/1/K/INF:

***

* *Número de usuarios*
\begin{equation}
NS = \lambda \times TS
\end{equation}

* *Tiempo en el sistema*
\begin{equation}
TS = \frac{1}{\mu - \lambda}
\end{equation}

* *Número de usuarios en la cola*
\begin{equation}
N_q = \frac{\lambda^2}{2\mu(\mu - \lambda)} - 1
\end{equation}

* *Tiempo en cola*
\begin{equation}
Ts = \frac{N_q}{\lambda}
\end{equation}

In [ ]:
!pip install simpy

In [ ]:
import simpy
import numpy as np

Se Define la clase para el servidor

In [ ]:
class Server:
    def __init__(self, env, mu, num_servers):
        self.env = env
        self.mu = mu
        self.servers = [simpy.Resource(env, capacity=1) for _ in range(num_servers)]

    def serve(self, service_time, server_id):
        yield self.env.timeout(service_time)

* Función para la llegada de usuarios al sistema
* Se genera tiempo hasta la próxima llegada
* Se crea un nuevo usuario

In [ ]:
def arrival(env, servers, lambd):
    while True:

        interarrival_time = np.random.exponential(1 / lambd)
        yield env.timeout(interarrival_time)

        env.process(user(env, servers))


* Función para el comportamiento de los usuarios
  * Se elige el servidor menos ocupado
  * Se espera a ser atendido por el servidor (yield request)
  * Se genera el tiempo de servicio
  * Se sirve al usuario
  * Se calcula tiempo en el sistema y tiempo en cola
  * Se registran las métricas

In [ ]:
def user(env, servers):
    arrival_time = env.now

    server_id = np.argmin([len(server.queue) for server in servers.servers])

    with servers.servers[server_id].request() as request:

        yield request

        service_time = np.random.exponential(1 / mu)

        yield env.process(servers.serve(service_time, server_id))

        time_in_system = env.now - arrival_time
        time_in_queue = time_in_system - service_time

        total_users_in_system.append(len(servers.servers[server_id].queue) + 1)
        total_time_in_system.append(time_in_system)
        total_users_in_queue.append(len(servers.servers[server_id].queue))
        total_time_in_queue.append(time_in_queue)


### Simulación del sistema M/M/1/INF/INF

In [ ]:
def simulate_mm1_inf_inf(lambd, mu, num_servers, sim_time):
    env = simpy.Environment()
    servers = Server(env, mu, num_servers)
    env.process(arrival(env, servers, lambd))
    env.run(until=sim_time)

### Simulación del sistema M/M/1/K/INF
* (num_servers) Se establece la capacidad de la cola

In [ ]:
def simulate_mm1_k_inf(lambd, mu, K, num_servers, sim_time):
    env = simpy.Environment()
    servers = Server(env, mu, num_servers)
    servers.servers = [simpy.Resource(env, capacity=K) for _ in range(num_servers)]
    env.process(arrival(env, servers, lambd))
    env.run(until=sim_time)

* Se pide al usuario el número de servidores a probar
* Se establecen los parametros de simulación
  * Donde:
    * lambd - tasa de llegada de usuarios
    * mu - tasa de servicio del servidor
    * K - tamaño máximo de la cola (INF para M/M/1/INF/INF)
    * sim_time - tiempo de simulación

* Luego tenemos
  * Las listas para registrar las metricas

* Simulación M/M/1/INF/INF
  * simulate_mm1_inf_inf


* Simulación M/M/1/K/INF
  * simulate_mm1_k_inf

* Se convierten tiempos medios negativos en cola a cero

In [ ]:
num_servers = int(input("Ingrese el número de servidores a probar: "))

lambd = 0.2
mu = 0.4
K = float('inf')
sim_time = 1000

total_users_in_system = []
total_time_in_system = []
total_users_in_queue = []
total_time_in_queue = []

simulate_mm1_inf_inf(lambd, mu, num_servers, sim_time)

print("Resultados para M/M/1/INF/INF con")
print("Número medio de usuarios en el sistema:", np.mean(total_users_in_system))
print("Tiempo medio en el sistema:", np.mean(total_time_in_system))
print("Número medio de usuarios en cola:", np.mean(total_users_in_queue))
print("Tiempo medio en cola:", np.mean(total_time_in_queue))

total_users_in_system = []
total_time_in_system = []
total_users_in_queue = []
total_time_in_queue = []

simulate_mm1_k_inf(lambd, mu, K, num_servers, sim_time)
total_time_in_queue[:] = [max(0, time) for time in total_time_in_queue]

print("\nResultados para M/M/1/K/INF (con K = INF) con", num_servers, "servidores:")
print("Número medio de usuarios en el sistema:", np.mean(total_users_in_system))
print("Tiempo medio en el sistema:", np.mean(total_time_in_system))
print("Número medio de usuarios en cola:", np.mean(total_users_in_queue))
print("Tiempo medio en cola:", np.mean(total_time_in_queue))

Ingrese el número de servidores a probar: 3
Resultados para M/M/1/INF/INF con 3
Número medio de usuarios en el sistema: 1.2681564245810055
Tiempo medio en el sistema: 3.0481893468859966
Número medio de usuarios en cola: 0.2681564245810056
Tiempo medio en cola: 0.6577821211401399

Resultados para M/M/1/K/INF (con K = INF) con 3 servidores:
Número medio de usuarios en el sistema: 1.0
Tiempo medio en el sistema: 2.6794100097444637
Número medio de usuarios en cola: 0.0
Tiempo medio en cola: 8.486735035078804e-15


* ¿Cómo se comporta el sistema si se aumenta el numero de servidores?


Al aumentar el número de servidores en el sistema, se observa que tanto el número medio de usuarios en el sistema como el tiempo medio en el sistema tienden a disminuir. Esto se debe a que hay más capacidad de servicio disponible para atender a los usuarios, lo que reduce la probabilidad de congestión y los tiempos de espera.

En cuanto al número medio de usuarios en cola y el tiempo medio en cola, también tienden a disminuir al aumentar el número de servidores, ya que hay más capacidad disponible para procesar las solicitudes de los usuarios en cola, lo que reduce los tiempos de espera en la cola.

Como conclusión, tenemos que al aumentar el número de servidores, el sistema tiende a volverse más eficiente y puede manejar un mayor volumen de usuarios con tiempos de espera más cortos y una menor congestión en la cola.